In [1]:
import pandas as pd
import string

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('1_Rating.csv')
# df2 = pd.read_csv('2_Rating.csv')
# df3 = pd.read_csv('3_Rating.csv')
# df4 = pd.read_csv('4_Rating.csv')
# df5 = pd.read_csv('5_Rating.csv')

df1.drop(columns=['Unnamed: 0'], inplace = True)
# df2.drop(columns=['Unnamed: 0'], inplace = True)
# df3.drop(columns=['Unnamed: 0'], inplace = True)
# df4.drop(columns=['Unnamed: 0'], inplace = True)
# df5.drop(columns=['Unnamed: 0'], inplace = True)

In [3]:
one_star_reviews = df1['Text'].tolist()
# two_star_reviews = df2['Text'].tolist()
# three_star_reviews = df3['Text'].tolist()
# four_star_reviews = df4['Text'].tolist()
# five_star_reviews = df5['Text'].tolist()

In [4]:
one_star_reviews[50]

'The Sugar Factory is known for it\'s delicious drinks and some of the worst service in town.\n\nSeriously, I\'ve had a problem with the Sugar Factory on the strip with customer service, and after visiting this location for the first time the other day, my hopes of better service crumbled, it\'s even worse here.\n\nWhile the hostess was great and quick about seating us, we sat there for about 12 minutes before anyone even came over. Not only was the waitress slow, she seemed confused as to what tables she was suppose to be waiting. A moment after she left, another waiter came to take our order once again, and was confused that we had ordered with someone else already.\n\nI usually get the veggie burger from Sugar Factory, not only is it really the only acceptable vegetarian dish on the menu, it\'s just delicious. I don\'t know whether they changed the recipe or the chef is an idiot, but the once signature purple bun was replaced with a regular white bun. Bummer. :(\n\nFinishing touch t

# Need to clean up the text in the reviews as I tokenize so my generator doesn't produce something like '\n'.

In [5]:
def cleanin_reviews(review):
    # Many reviews have \n characters through my scrape, so I need to replace those with empty spaces
    review = review.replace('\n\n', ' ')
    # Splitting on spaces to tokenize
    tokens = review.split()
    # Getting rid of punctuation because 'word.' will be considered different from 'word'
    
    tokens = [word.lower() for word in tokens]
    return tokens

In [6]:
def tokenizer(star):
    all_tokens = []

    for reviews in star:
        all_tokens.append(cleanin_reviews(reviews))
    
    # The above makes a list of lists, but I need to have one giant list of tokens
    flat_list = [item for sublist in all_tokens for item in sublist]
    return flat_list

# One Star Reviews

In [7]:
one_stars = tokenizer(one_star_reviews)

In [8]:
all_chars = []
for words in one_stars:
    for chars in words:
        all_chars.append(chars)

In [9]:
print('Total Tokens: %d' % len(all_chars))
print('Unique Tokens: %d' % len(set(all_chars)))

Total Tokens: 6087938
Unique Tokens: 183


In [10]:
chars = sorted(list(set(all_chars)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [11]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, len(all_chars) - seq_length, 1):
    seq_in = all_chars[i:i + seq_length]
    seq_out = all_chars[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  6087838


In [12]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(len(chars))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [13]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [15]:
model.fit(X, y, epochs=20, batch_size=256, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
   6656/6087838 [..............................] - ETA: 7:54:41 - loss: 3.7066

KeyboardInterrupt: 

# Two Star Reviews

In [ ]:
two_stars = tokenizer(two_star_reviews)

In [ ]:
print('Total Tokens: %d' % len(two_stars))
print('Unique Tokens: %d' % len(set(two_stars)))

# Three Star Reviews

In [ ]:
three_stars = tokenizer(three_star_reviews)

In [ ]:
print('Total Tokens: %d' % len(three_stars))
print('Unique Tokens: %d' % len(set(three_stars)))

# Four Star Reviews

The lower total tokens count represents the smaller datasets for the review set. If the model ends up not being as robust as the others, I will obtain additional datapoints.

In [ ]:
four_stars = tokenizer(four_star_reviews)

In [ ]:
print('Total Tokens: %d' % len(four_stars))
print('Unique Tokens: %d' % len(set(four_stars)))

# Five Star Reviews

The lower total tokens count represents the smaller datasets for the review set. If the model ends up not being as robust as the others, I will obtain additional datapoints.

In [ ]:
five_stars = tokenizer(five_star_reviews)

In [ ]:
print('Total Tokens: %d' % len(five_stars))
print('Unique Tokens: %d' % len(set(five_stars)))

In [ ]:
def sequence_of_tokens(stars):
    length = 5 + 1
    sequences = list()
    for i in range(length, len(stars)):
        seq = stars[i-length:i]

        line = ' '.join(seq)

        sequences.append(line)
    print('Total Sequences: %d' % len(sequences))
    return sequences

In [ ]:
one_star = sequence_of_tokens(one_stars)

In [ ]:
two_star = sequence_of_tokens(two_stars)

In [ ]:
three_star = sequence_of_tokens(three_stars)

In [ ]:
four_star = sequence_of_tokens(four_stars)

In [ ]:
five_star = sequence_of_tokens(five_stars)

# Need to save these different sequences down to train my model

In [ ]:
def saver(lines, name):
    data = '\n'.join(lines)
    file = open(name, 'w')
    file.write(data)
    file.close()

In [ ]:
saver(one_star, 'One_Star.txt')
saver(two_star, 'Two_Star.txt')
saver(three_star, 'Three_star.txt')
saver(four_star, 'Four_Star.txt')
saver(five_star, 'Five_Star.txt')